# Korzhinskii potential minimization (T, P, $\mu$O<sub>2</sub> constrained)

In [ ]:
import numpy as np
import scipy.optimize as opt
import scipy.linalg as lin 
import sys

In [ ]:
from thermoengine import core, phases, model, equilibrate

In [ ]:
np.set_printoptions(linewidth=200, precision=1)

## Create phases for equilibrium assemblages

In [ ]:
modelDB = model.Database(liq_mod='v1.0')

In [ ]:
Liquid = modelDB.get_phase('Liq')
Feldspar = modelDB.get_phase('Fsp')
Quartz = modelDB.get_phase('Qz')

The Berman model database provides the SWIM water model by default.  Instead, override that choice by instantiating the MELTS 1.0.2 water model directly.

In [ ]:
Water = phases.PurePhase('WaterMelts', 'H2O', calib=False)

## Define elements in system and phases in system

In [ ]:
elm_sys = ['H','O','Na','Mg','Al','Si','P','K','Ca','Ti','Cr','Mn','Fe','Co','Ni']
#phs_sys = [Liquid, Feldspar, Water, Quartz]
phs_sys = [Liquid]

## Composition of the system
This is a high-silica rhyolite

In [ ]:
grm_oxides = {
    'SiO2':  77.5, 
    'TiO2':   0.08, 
    'Al2O3': 12.5, 
    'Fe2O3':  0.207,
    'Cr2O3':  0.0, 
    'FeO':    0.473, 
    'MnO':    0.0,
    'MgO':    0.03, 
    'NiO':    0.0, 
    'CoO':    0.0,
    'CaO':    0.43, 
    'Na2O':   3.98, 
    'K2O':    4.88, 
    'P2O5':   0.0, 
    'H2O':    5.5
}
tot_grm_oxides = 0.0
for key in grm_oxides.keys():
    tot_grm_oxides += grm_oxides[key]

Cast this composition as moles of elements for input to the Equilibrate class

In [ ]:
mol_oxides = core.chem.format_mol_oxide_comp(grm_oxides, convert_grams_to_moles=True)
moles_end,oxide_res = Liquid.calc_endmember_comp(
    mol_oxide_comp=mol_oxides, method='intrinsic', output_residual=True)
if not Liquid.test_endmember_comp(moles_end):
    print ("Calculated composition is infeasible!")
mol_elm = Liquid.covert_endmember_comp(moles_end,output='moles_elements')

In [ ]:
blk_cmp = []
for elm in elm_sys:
    index = core.chem.PERIODIC_ORDER.tolist().index(elm)
    blk_cmp.append(mol_elm[index])
blk_cmp = np.array(blk_cmp)

## Function to constrain the chemical potential of O<sub>2</sub>

In [ ]:
def mu0O2(t, p):
    tr = 298.15
    hs = 23.10248*(t-tr) + 2.0*804.8876*(np.sqrt(t)-np.sqrt(tr)) - 1762835.0*(1.0/t-1.0/tr) \
       - 18172.91960*np.log(t/tr) + 0.5*0.002676*(t*t-tr*tr)
    ss = 205.15 + 23.10248*np.log(t/tr)  - 2.0*804.8876*(1.0/np.sqrt(t)-1.0/np.sqrt(tr)) \
       - 0.5*1762835.0*(1.0/(t*t)-1.0/(tr*tr)) + 18172.91960*(1.0/t-1.0/tr) + 0.002676*(t-tr)
    return hs - t*ss
def log10NNO(t, p):
    return -25018.7/t + 12.981 + 0.046*(p-1.0)/t - 0.5117*np.log(t)
def muNNO(t, p, delta=0.0):
    return 8.3144598*t*np.log(10.0)*(log10NNO(t, p) + delta)

In [ ]:
def kc_ferric_ferrous(t, p, m, mtype='components', compute='logfO2', deltaNNO=0.0):
    assert isinstance(m, np.ndarray), 'm must be an numpy array of length 15'
    assert m.size == 15, 'm must be a 1-d numpy array of length 15'
    assert mtype == 'components', 'the value of mtype must be "components"'
    assert compute == 'logfO2' or compute == 'chem_pot' or compute == 'oxides', \
    'the value of compute must be "logfO2" or "chem_pot" or "oxides"'
    t0 = 1673.15
    a =    0.196
    b =    1.1492e4
    c =   -6.675
    e =   -3.364
    f =   -7.01e-7  * 1.0e5
    g =   -1.54e-10 * 1.0e5
    h =    3.85e-17 * 1.0e5 * 1.0e5
    d = np.array([0, 0, -2.243, 0, 0, -1.828, 0, 0, 0, 0, 3.201, 5.854, 6.215, 0, 0])
    # 'SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'MgCr2O4', 'Fe2SiO4', 'MnSi0.5O2', 'Mg2SiO4', 
    # 'NiSi0.5O2', 'CoSi0.5O2', 'CaSiO3', 'Na2SiO3', 'KAlSiO4', 'Ca3(PO4)2', 'H2O'
    ox = np.array([m[0]+m[5]+m[6]/2.+m[7]+m[8]/2.+m[9]/2.+m[10]+m[11]+m[12], # SiO2
                   m[1],            # TiO2
                   m[2]+m[12]/2.,   # Al2O3
                   m[3],            # Fe2O3
                   m[4],            # Cr2O3
                   m[5]*2.,         # FeO
                   m[6],            # MnO
                   m[4]+m[7]*2.0,   # MgO
                   m[8],            # NiO
                   m[9],            # CoO
                   m[10]+m[13]*3.,  # CaO
                   m[11],           # Na2O
                   m[12]/2.,        # K2O
                   m[13],           # P2O5
                   m[14]            # H2O
                  ])
    tot = np.sum(ox) + m[3]
    if (m[3] == 0.0 and m[5] == 0.0) or (tot == 0.0):
        return 0.0
    if compute == 'logfO2' or compute == 'chem_pot':
        temp  = b/t + c + e*(1.0-t0/t - np.log(t/t0)) + f*p/t + g*(t-t0)*p/t + h*p*p/t
        temp += (np.dot(ox, d) + 2.0*d[5]*ox[3] - d[3]*ox[3])/tot
        logfo2 = (np.log(ox[3]/ox[5]) - temp)/(a*np.log(10.0))
        return logfo2 if compute == 'logfO2' else 8.3144598*t*np.log(10.0)*logfo2
    elif compute == 'oxides':
        ox[5] += 2.0*ox[3]
        ox[3] = 0.0
        logfO2 = log10NNO(t, p) + deltaNNO
        temp = a*np.log(10.0)*logfO2 + b/t + c + e*(1.0-t0/t - np.log(t/t0)) \
             + f*p/t + g*(t-t0)*p/t + h*p*p/t
        temp += np.dot(ox, d)/tot
        temp = np.exp(temp)
        ox[3]  = temp*ox[5]/(1.0 + 2.0*temp)
        ox[5] -= 2.0*ox[3]
        return ox
    else:
        return 0.0

In [ ]:
t = 1050.0
p = 1750.0

In [ ]:
mu = Liquid.chem_potential(t,p,mol=moles_end)[0]
mu_SiO2    = mu[0]
mu_Fe2O3   = mu[3]
mu_Fe2SiO4 = mu[5]
print ('log fO2 (delta NNO)', kc_ferric_ferrous(t, p, moles_end) - (-25018.7/t + 12.981 + 0.046*(p-1.0)/t - 0.5117*np.log(t)))
print ('mu O2 MELTS', 2.0*mu_Fe2O3 + 2.0*mu_SiO2 - 2.0*mu_Fe2SiO4 - mu0O2(t,p), 'mu NNO', muNNO(t,p), 'mu O2 KC', 
      kc_ferric_ferrous(t, p, moles_end, compute='chem_pot'))
com = kc_ferric_ferrous(t, p, moles_end, compute='oxides', deltaNNO=0.0)
print ('Input Fe2O3', mol_oxides[3]*core.chem.oxide_props['molwt'][3]*100/tot_grm_oxides)
print ('Input FeO  ', mol_oxides[5]*core.chem.oxide_props['molwt'][5]*100/tot_grm_oxides)
print ('Comp  Fe2O3', com[3]*core.chem.oxide_props['molwt'][3]*100/tot_grm_oxides)
print ('Comp  FeO  ', com[5]*core.chem.oxide_props['molwt'][5]*100/tot_grm_oxides)

## Instantiate class instance and run calculation

In [ ]:
def muO2(t, p, state):
    global NNO_offset
    global muO2_debug
    moles = state.phase_d['Liquid']['moles']
    mu = state.phase_d['Liquid']['obj'].chem_potential(t,p,mol=moles)[0]
    muO2 = 2.0*mu[3] + 2.0*mu[0] - 2.0*mu[5]
    muO2ex = muNNO(t,p, delta=NNO_offset)
    muO2kc = kc_ferric_ferrous(t, p, moles, compute='chem_pot')
    if muO2_debug:
        print ('cur muO2', muO2, 'imp muO2', muO2ex, 'kc muO2', muO2kc, 'err muO2', muO2ex-muO2kc)
    return muO2 - (muO2ex-muO2kc) 

In [ ]:
equil = equilibrate.Equilibrate(elm_sys, phs_sys, lagrange_l=[({'O':2.0},muO2)])

In [ ]:
t = 1050.0
p = 1750.0
NNO_offset = 0.0
muO2_debug = True
state = equil.execute(t, p, bulk_comp=blk_cmp, debug=0)
state.print_state()

In [ ]:
moles = state.phase_d['Liquid']['moles']
com = kc_ferric_ferrous(t, p, moles, compute='oxides', deltaNNO=NNO_offset)
oxides = state.oxide_comp('Liquid')
print ('Input Fe2O3', mol_oxides[3]*core.chem.oxide_props['molwt'][3]*100/tot_grm_oxides)
print ('Input FeO  ', mol_oxides[5]*core.chem.oxide_props['molwt'][5]*100/tot_grm_oxides)
print ('Comp  Fe2O3', oxides['Fe2O3'])
print ('Comp  FeO  ', oxides['FeO'])
print ('kc    Fe2O3', com[3]*core.chem.oxide_props['molwt'][3]*100/tot_grm_oxides)
print ('kc    FeO  ', com[5]*core.chem.oxide_props['molwt'][5]*100/tot_grm_oxides)

Pickup runs use previously computed state

In [ ]:
state = equil.execute(t-10.0, p, state=state, debug=0)
state.print_state()

In [ ]:
moles = state.phase_d['Liquid']['moles']
com = kc_ferric_ferrous(t-10.0, p, moles, compute='oxides', deltaNNO=NNO_offset)
oxides = state.oxide_comp('Liquid')
print ('Input Fe2O3', mol_oxides[3]*core.chem.oxide_props['molwt'][3]*100/tot_grm_oxides)
print ('Input FeO  ', mol_oxides[5]*core.chem.oxide_props['molwt'][5]*100/tot_grm_oxides)
print ('Comp  Fe2O3', oxides['Fe2O3'])
print ('Comp  FeO  ', oxides['FeO'])
print ('kc    Fe2O3', com[3]*core.chem.oxide_props['molwt'][3]*100/tot_grm_oxides)
print ('kc    FeO  ', com[5]*core.chem.oxide_props['molwt'][5]*100/tot_grm_oxides)

In [ ]:
state = equil.execute(t-100.0, p, state=state, debug=0)
state.print_state()

In [ ]:
moles = state.phase_d['Liquid']['moles']
com = kc_ferric_ferrous(t-100.0, p, moles, compute='oxides', deltaNNO=NNO_offset)
oxides = state.oxide_comp('Liquid')
print ('Input Fe2O3', mol_oxides[3]*core.chem.oxide_props['molwt'][3]*100/tot_grm_oxides)
print ('Input FeO  ', mol_oxides[5]*core.chem.oxide_props['molwt'][5]*100/tot_grm_oxides)
print ('Comp  Fe2O3', oxides['Fe2O3'])
print ('Comp  FeO  ', oxides['FeO'])
print ('kc    Fe2O3', com[3]*core.chem.oxide_props['molwt'][3]*100/tot_grm_oxides)
print ('kc    FeO  ', com[5]*core.chem.oxide_props['molwt'][5]*100/tot_grm_oxides)